# 第3回講義 宿題

## 課題
今Lessonで学んだことを元に、MNISTのファッション版 (Fashion MNIST、クラス数10) をソフトマックス回帰によって分類してみましょう。

Fashion MNISTの詳細については以下のリンクを参考にしてください。

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください。**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)。
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません。

### 提出方法
- 2つのファイルを提出していただきます。
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し、**Homeworkタブから`chap03`を選択して**提出してください。
    2. それに対応するpythonのコードを`submission_code.py`として保存し、**Homeworkタブから`chap03 (code)`を選択して**提出してください。
      
- なお、採点は1で行い、2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）。コードの内容を変更した場合は、**1と2の両方を提出し直してください**。

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します。
- 定時に採点しLeader Boardを更新します。(採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします。

### データの読み込み (このセルは修正しないでください)

In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

x_train = np.load('/home/sato.mizuki/deeplearningUT/Lecture_20210415/data/x_train.npy')
#drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210415/data/x_train.npy
def load_fashionmnist():
    # 学習データ
    x_train = np.load('/home/sato.mizuki/deeplearningUT/Lecture_20210415/data/x_train.npy')
    y_train = np.load('/home/sato.mizuki/deeplearningUT/Lecture_20210415/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('/home/sato.mizuki/deeplearningUT/Lecture_20210415/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### ソフトマックス回帰の実装

In [4]:

x_train, y_train, x_test = load_fashionmnist()

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)
    # WRITE ME

# weights
W = np.random.normal(0, 1/784, (784, 10)).astype('float32')
#np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')# WRITE ME
b = np.zeros(shape=(10,)).astype('float32')# WRITE ME

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1,random_state=1)

# logの中身が0になるのを防ぐ
def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def train(x, t, eps=1.0):
    global W, b
    batch_size = x.shape[0]

    #prediction
    y_hat = softmax(np.matmul(x, W)+b)

    #evaluation
    cost = (-t * np_log(y_hat)).sum(axis=1).mean()
    delta = y_hat - t

    #param
    dW = np.matmul(x.T, delta)
    db = np.matmul(np.ones(batch_size), delta)

    W -= eps*dW
    b -= eps*db

    return cost
    
    # WRITE ME

def valid(x, t):
    y_hat = softmax(np.matmul(x, W) + b)
    cost = (-t*np_log(y_hat)).sum(axis=1).mean()

    return cost, y_hat

    # WRITE ME
eps = 2.0
for epoch in range(5000):
    # オンライン学習
    pre_accuracy = 0
    x_train, y_train = shuffle(x_train, y_train, random_state=0)
    cost = train(x_train, y_train, eps)
    cost, y_pred = valid(x_valid, y_valid)
    accuracy = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    if epoch % 10 == 9 or epoch == 0:
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch+1,
            cost,
            accuracy
        ))

    if accuracy > 0.7:
        eps = 0.9
    elif accuracy >0.75:
        eps = 0.8
    elif accuracy >0.8:
        eps = 0.2
    else:
        eps = 1.0

    if accuracy > 0.85:
        print(accuracy)
        break
    # WRITE ME

y_pred = softmax(np.matmul(x_test, W) + b)
# WRITE ME
"""
submission = pd.Series(y_pred, name='label')
submission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210415/submission_pred.csv', header=True, index_label='id')
"""

EPOCH: 1, Valid Cost: 16.459, Valid Accuracy: 0.285
EPOCH: 10, Valid Cost: 10.703, Valid Accuracy: 0.535
EPOCH: 20, Valid Cost: 8.746, Valid Accuracy: 0.620
EPOCH: 30, Valid Cost: 7.987, Valid Accuracy: 0.653
EPOCH: 40, Valid Cost: 7.691, Valid Accuracy: 0.666
EPOCH: 50, Valid Cost: 6.129, Valid Accuracy: 0.734
EPOCH: 60, Valid Cost: 5.657, Valid Accuracy: 0.754
EPOCH: 70, Valid Cost: 7.169, Valid Accuracy: 0.689
EPOCH: 80, Valid Cost: 6.159, Valid Accuracy: 0.733
EPOCH: 90, Valid Cost: 6.547, Valid Accuracy: 0.716
EPOCH: 100, Valid Cost: 6.397, Valid Accuracy: 0.722
EPOCH: 110, Valid Cost: 5.895, Valid Accuracy: 0.744
EPOCH: 120, Valid Cost: 6.885, Valid Accuracy: 0.701
EPOCH: 130, Valid Cost: 5.446, Valid Accuracy: 0.763
EPOCH: 140, Valid Cost: 5.288, Valid Accuracy: 0.770
EPOCH: 150, Valid Cost: 5.637, Valid Accuracy: 0.755
EPOCH: 160, Valid Cost: 4.482, Valid Accuracy: 0.805
EPOCH: 170, Valid Cost: 5.787, Valid Accuracy: 0.749
EPOCH: 180, Valid Cost: 6.662, Valid Accuracy: 0.711
EP

"\nsubmission = pd.Series(y_pred, name='label')\nsubmission.to_csv('drive/My Drive/Colab Notebooks/DLBasics2021_colab/Lecture_20210415/submission_pred.csv', header=True, index_label='id')\n"